<a href="https://colab.research.google.com/github/Amitgm/LLm/blob/main/Fine%20Tuning/Zephyr%20Fine%20Tuning/Zephyr_Qlora_Fine_Tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

**IMPORTING THE NECSSARY LIBRARIES**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, PeftModel, PeftConfig, get_peft_model, prepare_model_for_kbit_training
import torch
from datasets import load_dataset
import os
import numpy as np
import transformers

**THE CONFIGURATIONS FOR TRAINING ARGUMENT AND LORA CONFIGURATIONS**

In [ ]:
class Config:
  model_name = "HuggingFaceH4/zephyr-7b-alpha"
  dataset_name = "SoorajK1/questions_and_answers"
  load_in_4bit=True
  bnb_4bit_quant_type="nf4"
  bnb_4bit_use_double_quant=True
  bnb_4bit_compute_dtype=torch.float16
  # LORA CONFIGURATIONS
  r=16
  lora_alpha=32
  lora_dropout=0.05
  target_modules=["q_proj", "v_proj"]
  bias="none"
  task_type="CAUSAL_LM"
  # TRAINING CONFIGURATIONS
  return_sequences = 1
  per_device_train_batch_size = 2
  epochs = 1
  learning_rate = 2e-4
  weight_decay = 0.001
  gradient_accumulation_steps = 4
  optim="paged_adamw_8bit",
  lr_scheduler_type="cosine"
  MAX_SEQ_LENGTH = 512
  output_dir ="Experiments"
  fp16=True
  warmup_ratio=0.05,
  report_to="tensorboard"



**ZEPHYR TRAINER CLASS DEFINED FOR THE ZEPHYR TRAINER**

In [ ]:
class Zephyr_trainer:

  def __init__(self):

    """
    TRAINER IS USED TO TRAIN THE ZEPHYR ALPHA MODEL FOR 7 BILLION PARAMETERS
    """
    device = "cuda:0"
    self.config = Config()

    MODEL_NAME = self.config.model_name

    bite_and_bytes_config = BitsAndBytesConfig(
        load_in_4bit = self.config.load_in_4bit,
        bnb_4bit_quant_type = self.config.bnb_4bit_quant_type,
        bnb_4bit_use_double_quant = self.config.bnb_4bit_use_double_quant,
        bnb_4bit_compute_dtype = self.config.bnb_4bit_compute_dtype,
    )


    self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)

    self.model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,device_map={"":0},trust_remote_code=True, quantization_config=bite_and_bytes_config)


  def dataset_preprocess(self,example):

    print(example["question"])
    print(example["answer"])

    processed_example = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.\n<|user|>\n" + example["question"] + "\n<|assistant|>\n" + example["answer"]

    tokenized_full_prompt = self.tokenizer(processed_example, padding=True, truncation=True)

    return tokenized_full_prompt

  def load_dataset(self):

    dataset = load_dataset(self.config.dataset_name)

    return dataset


  def model_train(self,data):


    lora_config = LoraConfig(
        r=self.config.r,
        lora_alpha=self.config.lora_alpha,
        target_modules=self.config.target_modules,
        lora_dropout=self.config.lora_dropout,
        bias="none",
        task_type="CAUSAL_LM"
    )

    self.model.gradient_checkpointing_enable()

    model = prepare_model_for_kbit_training(self.model)

    model = get_peft_model(model,lora_config)

    training_args = TrainingArguments(
        per_device_train_batch_size = self.config.per_device_train_batch_size,
        gradient_accumulation_steps = self.config.gradient_accumulation_steps,
        learning_rate = self.config.learning_rate,
        max_steps = 60,
        output_dir=self.config.output_dir,
        lr_scheduler_type = self.config.lr_scheduler_type,
        num_train_epochs = self.config.epochs,
        logging_steps=1,
        optim = "paged_adamw_8bit",
        save_total_limit = 3,
        fp16=self.config.fp16,
        report_to = self.config.report_to

    )

    trainer = Trainer(
        model=model,
        train_dataset=data,
        args=training_args,
        data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=self.tokenizer, mlm=False)
    )

    trainer.train()

    return model



In [ ]:
zephyr_train = Zephyr_trainer()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

**LOADING THE DATASET**

In [ ]:
# LOADING THE DATASET
dataset = zephyr_train.load_dataset()

Generating train split:   0%|          | 0/29438 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2830 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2830 [00:00<?, ? examples/s]

In [ ]:
data= dataset["train"].shuffle(seed=42).map(zephyr_train.dataset_preprocess)

Map:   0%|          | 0/29438 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Streaming output truncated to the last 5000 lines.
Yes, phosphorus (DAP or SSP) application can be omitted in cotton if the preceding crop was wheat and it had received the recommended dose of phosphorus.
What type of devices are used to collect the data?
Smartphones are used to collect the data.
How can the fertility of soil be depleted?
The fertility of soil can be depleted through various factors and practices. Continuous cultivation of crops without adequate replenishment of organic matter or nutrients can exhaust the soil's fertility over time. Erosion, which can occur due to wind or water, can also wash away topsoil that contains essential nutrients. Additionally, the excessive use of chemical fertilizers, pesticides, and herbicides can disrupt the natural balance of soil microorganisms and reduce soil fertility in the long run. Soil compaction, caused by heavy machinery or livestock trampling, can also negatively impact soil fertility by reducing aeration and impairing root grow

**TRAINING THE ZEPHYR MODEL**

In [ ]:
model = zephyr_train.model_train(data)

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.232700
2,1.992500
3,1.746500
4,1.691100
5,1.473800
6,1.429400
7,1.078900
8,1.108300
9,0.964800
10,0.938800


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

**PUSHING THE ZEPHYR MODEL**

In [ ]:
model.push_to_hub(
    "Amitgm/Zephyr-qlora-chat-question-answer", use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Amitgm/Zephyr-qlora-chat-question-answer/commit/6b78107f9dfa691703bea01d1bc69cc0ee04cc4a', commit_message='Upload model', commit_description='', oid='6b78107f9dfa691703bea01d1bc69cc0ee04cc4a', pr_url=None, pr_revision=None, pr_num=None)

**LOADING THE MODEL ALONG WITH ITS ADAPTERS**

In [ ]:
PEFT_MODEL = "Amitgm/Zephyr-qlora-chat-question-answer"

In [ ]:
config = Config()

bite_and_bytes_config = BitsAndBytesConfig(
        load_in_4bit = config.load_in_4bit,
        bnb_4bit_quant_type = config.bnb_4bit_quant_type,
        bnb_4bit_use_double_quant = config.bnb_4bit_use_double_quant,
        bnb_4bit_compute_dtype = config.bnb_4bit_compute_dtype,
    )

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bite_and_bytes_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [ ]:
generation_config = model.generation_config

In [ ]:
config.base_model_name_or_path

'HuggingFaceH4/zephyr-7b-alpha'

**DEFINING THE GENERATION CONFIG ARGUMENTS**

In [ ]:
# MAXIMUM NEW TOKENS TO BE GENERATED
generation_config.max_new_tokens = 200
# WETHER THE MODEL NEEDS TO BE CREATIVE
generation_config.temperature = 0.1
# NUMBER OF OUTPUT SEQUENCES TO BE RETURNED
generation_config.num_return_sequences = 1
# GENERATE SEQUENCE UNTIL YOU GET TO THE END OF SEQUENCE TOKEN ID
generation_config.eos_token_id = tokenizer.eos_token_id
# PAD THE INPUT SEQUENCES WITH THE PAD TOKEN ID
generation_config.pad_token_id = tokenizer.pad_token_id

In [ ]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

In [ ]:
device = "cuda:0"

**GENERATE RESPONSE USING THE PROMPT TEMPLATE**

In [ ]:
def generate_reponse(question):

    prompt = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.\n<|user|>\n" + question + "\n<|assistant|>\n"

    encoding = tokenizer(prompt, return_tensors="pt").to(device)

    # with torch.inference_mode():

    outputs = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask,generation_config=generation_config)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    response_start = response.find("<|assistant|>")

    response_full = response[response_start+len("<|assistant|>"):].strip()

    return(response_full)

In [ ]:
generate_reponse("What is the purpose of the Android App mentioned in the text??")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'The purpose of the Android App mentioned in the text is to provide a user-friendly interface for the user to interact with the system. It allows the user to easily navigate through the system and access the information they need. The app is designed to be intuitive and easy to use, making it accessible to a wide range of users.'